In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset
from collections import OrderedDict
import numpy as np
from torchvision.models import vgg16
from codecarbon import EmissionsTracker


tracker = EmissionsTracker()
tracker.start()



def get_vgg16_model(num_classes):
    model = vgg16(pretrained=True)
    for param in model.parameters():
        param.requires_grad = False
    num_features = model.classifier[6].in_features
    model.classifier[6] = nn.Linear(num_features, num_classes)
    return model



def train_model(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = nn.CrossEntropyLoss()(output, target)
        loss.backward()
        optimizer.step()

def test_model(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += nn.CrossEntropyLoss(reduction='sum')(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    return test_loss / len(test_loader.dataset), correct / len(test_loader.dataset)

def average_weights(weight_list):
    avg_weights = OrderedDict()
    for key in weight_list[0].keys():
        avg_weights[key] = sum([client_weights[key] for client_weights in weight_list]) / len(weight_list)
    return avg_weights
    
def create_client_dataset(dataset, num_clients):
    indices = list(range(len(dataset)))
    np.random.shuffle(indices)
    client_indices = np.array_split(indices, num_clients)
    client_datasets = [Subset(dataset, index_list) for index_list in client_indices]
    return client_datasets

# Load the CIFAR-10 dataset
transform = transforms.Compose([transforms.RandomHorizontalFlip(), transforms.RandomCrop(32, padding=4), transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

num_clients = 5
client_datasets = create_client_dataset(train_dataset, num_clients)
train_loaders = [DataLoader(client_dataset, batch_size=100, shuffle=True) for client_dataset in client_datasets]
test_loader = DataLoader(test_dataset, batch_size=100, shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
epochs = 10


def federated_learning(train_loaders, test_loader, device, epochs, num_clients, communication_rounds, client_weighting, clients_per_round):
    global_model = get_vgg16_model(10).to(device)
    client_models = [get_vgg16_model(10).to(device) for _ in range(num_clients)]

    for client_model in client_models:
        client_model.load_state_dict(global_model.state_dict())

    for com_round in range(1, communication_rounds + 1):
        print(f'Communication round {com_round}/{communication_rounds}')

        selected_clients = np.random.choice(range(num_clients), size=clients_per_round, replace=False)
        client_weights = []

        for idx in selected_clients:
            client_model = client_models[idx]
            optimizer = optim.SGD(client_model.parameters(), lr=0.01, momentum=0.9)
            train_model(client_model, device, train_loaders[idx], optimizer, epochs)

            client_weight = client_weighting[idx] if client_weighting else 1
            weighted_client_state = OrderedDict()

            for key in client_model.state_dict().keys():
                weighted_client_state[key] = client_model.state_dict()[key] * client_weight

            client_weights.append(weighted_client_state)

        global_weights = average_weights(client_weights)
        global_model.load_state_dict(global_weights)

        test_loss, accuracy = test_model(global_model, device, test_loader)
        print(f'Test loss: {test_loss:.4f}, Accuracy: {accuracy * 100:.2f}%\n')

    return global_model

communication_rounds = 50
client_weighting = [1, 1, 1, 1, 1]  
clients_per_round = 5

federated_model = federated_learning(train_loaders, test_loader, device, epochs, num_clients, communication_rounds, client_weighting, clients_per_round)


tracker.stop()


[codecarbon INFO @ 11:19:08] [setup] RAM Tracking...
[codecarbon INFO @ 11:19:08] [setup] GPU Tracking...
[codecarbon INFO @ 11:19:08] No GPU found.
[codecarbon INFO @ 11:19:08] [setup] CPU Tracking...
[codecarbon WARNING @ 11:19:08] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 11:19:09] CPU Model on constant consumption mode: Apple M1
[codecarbon INFO @ 11:19:09] >>> Tracker's metadata:
[codecarbon INFO @ 11:19:09]   Platform system: macOS-10.16-x86_64-i386-64bit
[codecarbon INFO @ 11:19:09]   Python version: 3.8.16
[codecarbon INFO @ 11:19:09]   Available RAM : 8.000 GB
[codecarbon INFO @ 11:19:09]   CPU count: 8
[codecarbon INFO @ 11:19:09]   CPU model: Apple M1
[codecarbon INFO @ 11:19:09]   GPU count: None
[codecarbon INFO @ 11:19:09]   GPU model: None


Files already downloaded and verified
Files already downloaded and verified


/Users/jessikamakinen/opt/anaconda3/envs/FLthesis/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/Users/jessikamakinen/opt/anaconda3/envs/FLthesis/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /Users/jessikamakinen/.cache/torch/hub/checkpoints/vgg16-397923af.pth
4.1%[codecarbon INFO @ 11:19:28] Energy consumed for RAM : 0.000013 kWh. RAM Power : 3.0 W
4.1%[codecarbon INFO @ 11:19:28] Energy consumed for all CPUs : 0.000021 kWh. All C

Communication round 1/50


[W NNPACK.cpp:51] Could not initialize NNPACK! Reason: Unsupported hardware.
[codecarbon INFO @ 11:22:13] Energy consumed for RAM : 0.000150 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 11:22:13] Energy consumed for all CPUs : 0.000250 kWh. All CPUs Power : 5.0 W
[codecarbon INFO @ 11:22:13] 0.000400 kWh of electricity used since the begining.
[codecarbon INFO @ 11:22:28] Energy consumed for RAM : 0.000163 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 11:22:28] Energy consumed for all CPUs : 0.000271 kWh. All CPUs Power : 5.0 W
[codecarbon INFO @ 11:22:28] 0.000433 kWh of electricity used since the begining.
[codecarbon INFO @ 11:22:43] Energy consumed for RAM : 0.000175 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 11:22:43] Energy consumed for all CPUs : 0.000292 kWh. All CPUs Power : 5.0 W
[codecarbon INFO @ 11:22:43] 0.000467 kWh of electricity used since the begining.
[codecarbon INFO @ 11:22:58] Energy consumed for RAM : 0.000188 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 11:22:58] Energy co

...

[codecarbon INFO @ 15:10:55] Energy consumed for RAM : 0.011581 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 15:10:55] Energy consumed for all CPUs : 0.019307 kWh. All CPUs Power : 5.0 W
[codecarbon INFO @ 15:10:55] 0.030888 kWh of electricity used since the begining.
[codecarbon INFO @ 15:11:10] Energy consumed for RAM : 0.011594 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 15:11:10] Energy consumed for all CPUs : 0.019328 kWh. All CPUs Power : 5.0 W
[codecarbon INFO @ 15:11:10] 0.030921 kWh of electricity used since the begining.
[codecarbon INFO @ 15:11:25] Energy consumed for RAM : 0.011606 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 15:11:25] Energy consumed for all CPUs : 0.019349 kWh. All CPUs Power : 5.0 W
[codecarbon INFO @ 15:11:25] 0.030955 kWh of electricity used since the begining.
[codecarbon INFO @ 15:11:40] Energy consumed for RAM : 0.011619 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 15:11:40] Energy consumed for all CPUs : 0.019369 kWh. All CPUs Power : 5.0 W
[codecarbon INFO @

Test loss: 1.6853, Accuracy: 54.17%

Communication round 11/50


[codecarbon INFO @ 15:38:25] Energy consumed for RAM : 0.012955 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 15:38:25] Energy consumed for all CPUs : 0.021598 kWh. All CPUs Power : 5.0 W
[codecarbon INFO @ 15:38:25] 0.034553 kWh of electricity used since the begining.
[codecarbon INFO @ 15:38:40] Energy consumed for RAM : 0.012968 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 15:38:40] Energy consumed for all CPUs : 0.021618 kWh. All CPUs Power : 5.0 W
[codecarbon INFO @ 15:38:40] 0.034586 kWh of electricity used since the begining.
[codecarbon INFO @ 15:38:55] Energy consumed for RAM : 0.012980 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 15:38:55] Energy consumed for all CPUs : 0.021639 kWh. All CPUs Power : 5.0 W
[codecarbon INFO @ 15:38:55] 0.034619 kWh of electricity used since the begining.


KeyboardInterrupt: 

[codecarbon INFO @ 15:39:10] Energy consumed for RAM : 0.012993 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 15:39:10] Energy consumed for all CPUs : 0.021660 kWh. All CPUs Power : 5.0 W
[codecarbon INFO @ 15:39:10] 0.034653 kWh of electricity used since the begining.
